In [2]:
from Input_Data import Tanners_Clean_Code as TCC
from country_cleaner import country_clean

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import seaborn as sns

import statsmodels.api as sm
import statsmodels.formula.api as smf
from causalinference import CausalModel

import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from scipy import stats
import linearmodels

cwurData = "Input_Data/cwurData.csv" ##200 Nans in broad_impact column, 2200 rows/14 columns
td = "Input_Data/timesData.csv"##418 Nans, 4 columns have Nans, 2603 rows, 14 columns

cwur_df = pd.read_csv(cwurData)
td_df = pd.read_csv(td)


all_Expend = TCC.complete_expenditures_all
expend_K12 =  TCC.complete_expenditures_k_12
expend_Higher_ED = TCC.complete_expenditures_higher_edu

#final data-set with total GDP (not expenditures) to run test on as it will have more datapoints vs above compplete tables
GDP_df = TCC.complete_GDP

GDP_df['country'] = GDP_df['country'].apply(lambda x: country_clean(x))

In [3]:
unique_Names = GDP_df['country'].unique()
unique_Names

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Argentina',
       'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas, The',
       'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium',
       'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia',
       'Bosnia and Herzegovina', 'Botswana', 'Brazil',
       'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi',
       'Cambodia', 'Cameroon', 'Canada', 'Cayman Islands',
       'Central African Republic', 'Chile', 'China', 'Colombia',
       'Congo, Dem. Rep.', 'Congo, Rep.', 'Costa Rica', "Cote d'Ivoire",
       'Croatia', 'Cuba', 'Cyprus', 'Czech Republic', 'Denmark',
       'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Estonia',
       'Ethiopia', 'Fiji', 'Finland', 'France', 'Gabon', 'Gambia, The',
       'Georgia', 'Germany', 'Ghana', 'Greece', 'Guatemala', 'Guyana',
       'Haiti', 'Honduras', 'Hong Kong SAR, China', 'Hungary', 'Iceland',
       'India', 'Indonesia', 'Iran', 'Iraq', 'Ireland', 'Isr

In [4]:
all_Expend.head()

,country,series_name,Keep or not,1985,1986,1987,1990,1991,1992,1993,...,2013,2015,avg_2005_2015,Country Code,Indicator Name,Indicator Code,avg_2000_2011,institute_type,school_spend_2000_2011,school_expenditures
62,Australia,"Barro-Lee: Average years of total schooling, a...",1.0,11.05,NaN,NaN,11.06,NaN,NaN,NaN,...,NaN,NaN,11.485,AUS,GDP (current US$),NY.GDP.MKTP.CD,7.580678e+11,All Institutions,0.058,4.396793e+10
63,Australia,"Barro-Lee: Average years of total schooling, a...",1.0,11.20,NaN,NaN,11.18,NaN,NaN,NaN,...,NaN,NaN,11.460,AUS,GDP (current US$),NY.GDP.MKTP.CD,7.580678e+11,All Institutions,0.058,4.396793e+10
64,Australia,"Barro-Lee: Average years of total schooling, a...",1.0,10.84,NaN,NaN,11.04,NaN,NaN,NaN,...,NaN,NaN,11.645,AUS,GDP (current US$),NY.GDP.MKTP.CD,7.580678e+11,All Institutions,0.058,4.396793e+10
65,Australia,"Barro-Lee: Average years of total schooling, a...",1.0,11.02,NaN,NaN,11.20,NaN,NaN,NaN,...,NaN,NaN,11.655,AUS,GDP (current US$),NY.GDP.MKTP.CD,7.580678e+11,All Institutions,0.058,4.396793e+10
66,Australia,Barro-Lee: Percentage of population age 15+ wi...,1.0,1.12,NaN,NaN,1.09,NaN,NaN,NaN,...,NaN,NaN,0.815,AUS,GDP (current US$),NY.GDP.MKTP.CD,7.580678e+11,All Institutions,0.058,4.396793e+10


In [5]:
expend_K12.head()

,country,series_name,Keep or not,1985,1986,1987,1990,1991,1992,1993,...,2013,2015,avg_2005_2015,Country Code,Indicator Name,Indicator Code,avg_2000_2011,institute_type,school_spend_2000_2011,school_expenditures
62,Australia,"Barro-Lee: Average years of total schooling, a...",1.0,11.05,NaN,NaN,11.06,NaN,NaN,NaN,...,NaN,NaN,11.485,AUS,GDP (current US$),NY.GDP.MKTP.CD,7.580678e+11,Elementary and Secondary Institutions,0.041,3.108078e+10
63,Australia,"Barro-Lee: Average years of total schooling, a...",1.0,11.20,NaN,NaN,11.18,NaN,NaN,NaN,...,NaN,NaN,11.460,AUS,GDP (current US$),NY.GDP.MKTP.CD,7.580678e+11,Elementary and Secondary Institutions,0.041,3.108078e+10
64,Australia,"Barro-Lee: Average years of total schooling, a...",1.0,10.84,NaN,NaN,11.04,NaN,NaN,NaN,...,NaN,NaN,11.645,AUS,GDP (current US$),NY.GDP.MKTP.CD,7.580678e+11,Elementary and Secondary Institutions,0.041,3.108078e+10
65,Australia,"Barro-Lee: Average years of total schooling, a...",1.0,11.02,NaN,NaN,11.20,NaN,NaN,NaN,...,NaN,NaN,11.655,AUS,GDP (current US$),NY.GDP.MKTP.CD,7.580678e+11,Elementary and Secondary Institutions,0.041,3.108078e+10
66,Australia,Barro-Lee: Percentage of population age 15+ wi...,1.0,1.12,NaN,NaN,1.09,NaN,NaN,NaN,...,NaN,NaN,0.815,AUS,GDP (current US$),NY.GDP.MKTP.CD,7.580678e+11,Elementary and Secondary Institutions,0.041,3.108078e+10


In [6]:
pd.set_option('display.max_colwidth', None)
expend_Higher_ED = expend_Higher_ED[expend_Higher_ED['series_name'] == 'Barro-Lee: Percentage of population age 15+ with no education']

expend_Higher_ED.head(200) 

,country,series_name,Keep or not,1985,1986,1987,1990,1991,1992,1993,...,2013,2015,avg_2005_2015,Country Code,Indicator Name,Indicator Code,avg_2000_2011,institute_type,school_spend_2000_2011,school_expenditures
66,Australia,Barro-Lee: Percentage of population age 15+ with no education,1.0,1.12,NaN,NaN,1.09,NaN,NaN,NaN,...,NaN,NaN,0.815,AUS,GDP (current US$),NY.GDP.MKTP.CD,7.580678e+11,Higher Education Institutions,0.016,1.212909e+10
88,Austria,Barro-Lee: Percentage of population age 15+ with no education,1.0,0.47,NaN,NaN,0.95,NaN,NaN,NaN,...,NaN,NaN,1.310,AUT,GDP (current US$),NY.GDP.MKTP.CD,3.220216e+11,Higher Education Institutions,0.015,4.830324e+09
162,Belgium,Barro-Lee: Percentage of population age 15+ with no education,1.0,4.48,NaN,NaN,4.68,NaN,NaN,NaN,...,NaN,NaN,4.660,BEL,GDP (current US$),NY.GDP.MKTP.CD,3.899835e+11,Higher Education Institutions,0.014,5.459769e+09
293,Chile,Barro-Lee: Percentage of population age 15+ with no education,1.0,5.35,NaN,NaN,4.57,NaN,NaN,NaN,...,NaN,NaN,2.980,CHL,GDP (current US$),NY.GDP.MKTP.CD,1.389673e+11,Higher Education Institutions,0.024,3.335216e+09
421,Czech Republic,Barro-Lee: Percentage of population age 15+ with no education,1.0,1.73,NaN,NaN,1.90,NaN,NaN,NaN,...,NaN,NaN,0.210,CZE,GDP (current US$),NY.GDP.MKTP.CD,1.498510e+11,Higher Education Institutions,0.014,2.097914e+09
430,Denmark,Barro-Lee: Percentage of population age 15+ with no education,1.0,2.32,NaN,NaN,0.11,NaN,NaN,NaN,...,NaN,NaN,2.380,DNK,GDP (current US$),NY.GDP.MKTP.CD,2.653692e+11,Higher Education Institutions,0.019,5.042016e+09
478,Estonia,Barro-Lee: Percentage of population age 15+ with no education,1.0,2.56,NaN,NaN,1.88,NaN,NaN,NaN,...,NaN,NaN,0.200,EST,GDP (current US$),NY.GDP.MKTP.CD,1.513248e+10,Higher Education Institutions,0.017,2.572521e+08
499,Finland,Barro-Lee: Percentage of population age 15+ with no education,1.0,1.03,NaN,NaN,0.11,NaN,NaN,NaN,...,NaN,NaN,0.155,FIN,GDP (current US$),NY.GDP.MKTP.CD,2.085418e+11,Higher Education Institutions,0.019,3.962295e+09
521,France,Barro-Lee: Percentage of population age 15+ with no education,1.0,5.42,NaN,NaN,8.00,NaN,NaN,NaN,...,NaN,NaN,1.475,FRA,GDP (current US$),NY.GDP.MKTP.CD,2.206148e+12,Higher Education Institutions,0.015,3.309223e+10
558,Germany,Barro-Lee: Percentage of population age 15+ with no education,1.0,5.10,NaN,NaN,5.08,NaN,NaN,NaN,...,NaN,NaN,3.215,DEU,GDP (current US$),NY.GDP.MKTP.CD,2.899084e+12,Higher Education Institutions,0.013,3.768809e+10


In [7]:
len(unique_Names)

161

In [8]:
GDP_df.describe()

,Keep or not,1985,1986,1987,1990,1991,1992,1993,1995,1996,...,2007,2008,2009,2010,2011,2012,2013,2015,avg_2005_2015,avg_2000_2015
count,1777.0,858.000000,0.0,0.0,858.000000,0.0,0.0,0.0,858.000000,0.0,...,12.000000,28.000000,34.000000,1011.000000,104.000000,174.000000,399.000000,0.0,1777.000000,1.777000e+03
mean,1.0,13.526620,NaN,NaN,12.881946,NaN,NaN,NaN,12.325443,NaN,...,13.204613,7.521989,10.876122,10.490873,12.367088,8.951760,10.502137,NaN,10.583915,4.980685e+11
std,0.0,19.355474,NaN,NaN,17.653940,NaN,NaN,NaN,16.041286,NaN,...,9.539355,11.427528,13.661371,12.004613,11.374484,10.256898,9.976462,NaN,11.675231,1.699744e+12
min,1.0,0.050000,NaN,NaN,0.070000,NaN,NaN,NaN,0.100000,NaN,...,0.000000,0.000000,0.025840,0.000000,0.135670,0.000000,0.000000,NaN,0.000000,3.181619e+08
25%,1.0,3.265000,NaN,NaN,3.787500,NaN,NaN,NaN,4.162500,NaN,...,7.993745,0.584940,1.380535,3.856680,2.927678,0.837830,0.978110,NaN,2.590000,1.213449e+10
50%,1.0,6.750000,NaN,NaN,7.215000,NaN,NaN,NaN,7.575000,NaN,...,13.634825,2.205800,7.159275,8.160000,9.118965,4.419875,8.836470,NaN,8.020000,4.254428e+10
75%,1.0,9.997500,NaN,NaN,10.235000,NaN,NaN,NaN,10.425000,NaN,...,18.171118,10.634692,14.654363,11.325000,18.400253,13.261627,17.388794,NaN,12.598400,2.491082e+11
max,1.0,98.190000,NaN,NaN,95.640000,NaN,NaN,NaN,91.580000,NaN,...,27.880040,40.402380,52.940270,81.370000,50.623320,47.319160,44.419550,NaN,80.600000,1.407434e+13


In [9]:
GDP_df.head()

,country,series_name,Keep or not,1985,1986,1987,1990,1991,1992,1993,...,2010,2011,2012,2013,2015,avg_2005_2015,Country Code,Indicator Name,Indicator Code,avg_2000_2015
0,Afghanistan,"Barro-Lee: Average years of total schooling, age 15+, female",1.0,0.50,NaN,NaN,0.66,NaN,NaN,NaN,...,1.97,NaN,NaN,NaN,NaN,1.620,AFG,GDP (current US$),NY.GDP.MKTP.CD,1.232844e+10
1,Afghanistan,"Barro-Lee: Average years of total schooling, age 15+, total",1.0,1.57,NaN,NaN,1.87,NaN,NaN,NaN,...,3.85,NaN,NaN,NaN,NaN,3.530,AFG,GDP (current US$),NY.GDP.MKTP.CD,1.232844e+10
2,Afghanistan,"Barro-Lee: Average years of total schooling, age 25+, female",1.0,0.28,NaN,NaN,0.42,NaN,NaN,NaN,...,1.16,NaN,NaN,NaN,NaN,1.080,AFG,GDP (current US$),NY.GDP.MKTP.CD,1.232844e+10
3,Afghanistan,"Barro-Lee: Average years of total schooling, age 25+, total",1.0,1.16,NaN,NaN,1.49,NaN,NaN,NaN,...,3.23,NaN,NaN,NaN,NaN,2.915,AFG,GDP (current US$),NY.GDP.MKTP.CD,1.232844e+10
4,Afghanistan,Barro-Lee: Percentage of population age 15+ with no education,1.0,77.80,NaN,NaN,75.04,NaN,NaN,NaN,...,58.38,NaN,NaN,NaN,NaN,62.055,AFG,GDP (current US$),NY.GDP.MKTP.CD,1.232844e+10


In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

cwur_df = pd.read_csv(cwurData)
cwur_df = cwur_df[cwur_df['broad_impact'].notna()]

cwur_df = cwur_df.drop(['year'], axis=1)
cwur_df =cwur_df.drop(['institution'], axis=1)

encoder = LabelEncoder()
cwur_df['country_encoded'] = encoder.fit_transform(cwur_df['country'])

first_column = cwur_df.pop('country')
  
# insert column using insert(position,column_name,
# first_column) function
cwur_df.insert(0, 'country', first_column)

#cwur_df1 = cwur_df.groupby('country').mean().sort_values(by=['quality_of_education'],ascending=True)
cwur_df.head(200)


#Naive Bayes
##How would I use this with the data?

#Nearest Neighbors
##NO

##Gaussian

#Neural Net
#Least Squares
#Subset Lasso
#Gradient Boost Tree

,country,world_rank,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,country_encoded
200,USA,1,1,1,1,1,1,1,1,1.0,2,100.00,54
201,USA,2,2,11,2,4,5,3,3,4.0,6,99.09,54
202,USA,3,3,3,11,2,15,2,2,2.0,1,98.69,54
203,United Kingdom,4,1,2,10,5,10,9,12,13.0,48,97.64,57
204,United Kingdom,5,2,7,12,10,11,12,11,12.0,16,97.51,57
...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,USA,196,85,243,96,97,220,213,220,234.0,149,48.02,54
396,Denmark,197,3,339,478,195,240,257,200,238.0,48,47.98,13
397,Spain,198,2,86,55,206,213,309,406,339.0,204,47.97,48
398,Germany,199,12,355,109,194,215,150,187,161.0,426,47.97,18


In [11]:
#Naive Bayes